In [2]:
%load_ext autoreload
%autoreload 2


In [5]:

import torch
from torch.utils.data import Dataset
from torchvision import transforms

from helperFunctions import parse_name, rotation_matrix, classes, eps
from axisAngle import get_y, get_R
from quaternion import get_y as get_quaternion

from PIL import Image
import numpy as np
import scipy.io as spio
from scipy.spatial.distance import cdist
import os
import pickle

from dataGenerators import ImagesAll
from helperFunctions import parse_name, rotation_matrix
from axisAngle import get_y

import numpy as np
from sklearn.cluster import KMeans
import pickle
import progressbar
import sys



In [6]:
image_path = 'data/original'

# relevant variables
num_clusters = 50
print('num_clusters: ', num_clusters)
kmeans_file = 'data/kmeans_dictionary_axis_angle_' + str(num_clusters) + '.pkl'

# setup data loader to access the images
train_data = ImagesAll(image_path, 'real')
image_names = np.concatenate(train_data.list_image_names)

# get pose targets from training data
bar = progressbar.ProgressBar()
ydata = []
for i in bar(range(len(image_names))):
	image_name = image_names[i]
	_, _, az, el, ct, _ = parse_name(image_name)
	R = rotation_matrix(az, el, -ct)
	y = get_y(R)
	ydata.append(y)
ydata = np.stack(ydata)
print('\nData size: ', ydata.shape)

# run kmeans
kmeans = KMeans(num_clusters, verbose=1, n_jobs=10)
kmeans.fit(ydata)
print(kmeans.cluster_centers_)

# save output
fid = open(kmeans_file, 'wb')
pickle.dump(kmeans, fid)

del kmeans

# load and check
kmeans = pickle.load(open(kmeans_file, 'rb'))
print(kmeans.cluster_centers_)


num_clusters:  50
{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Mon Feb 25 17:21:46 2019', '__version__': '1.0', '__globals__': [], 'imagenet_train': array(['n02690373_10032', 'n02690373_10061', 'n02690373_101',
       'n02690373_102', 'n02690373_10203', 'n02690373_10308',
       'n02690373_1038', 'n02690373_10395', 'n02690373_1074',
       'n02690373_1087', 'n02690373_10898', 'n02690373_1098',
       'n02690373_1119', 'n02690373_1131', 'n02690373_1134',
       'n02690373_11369', 'n02690373_11488', 'n02690373_1151',
       'n02690373_1161', 'n02690373_1171', 'n02690373_11908',
       'n02690373_12044', 'n02690373_1206', 'n02690373_1213',
       'n02690373_1233', 'n02690373_1235', 'n02690373_12505',
       'n02690373_1279', 'n02690373_13045', 'n02690373_1341',
       'n02690373_1350', 'n02690373_1381', 'n02690373_1396',
       'n02690373_1416', 'n02690373_1436', 'n02690373_1474',
       'n02690373_1491', 'n02690373_1495', 'n02690373_1506',
       'n02690373_15392'

KeyError: 'image_names'